In [1]:
pip install tensorflow

In [23]:
import os
import gdown
import tarfile
import zipfile
import tensorflow as tf
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


In [3]:
file_id = '1ZxhBdiosr3KM7QyXT54B6XXi7VLuqJPH'
file_url = f'https://drive.google.com/uc?id={file_id}'

In [4]:
destination_dir = '/content/images'
os.makedirs(destination_dir, exist_ok=True)

In [5]:
zip_file_path = os.path.join(destination_dir, 'images.zip')
gdown.download(file_url, zip_file_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1ZxhBdiosr3KM7QyXT54B6XXi7VLuqJPH
To: /content/images/images.zip
100%|██████████| 354M/354M [00:07<00:00, 45.7MB/s]


'/content/images/images.zip'

In [7]:
if zip_file_path.endswith('.zip'):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_dir)
elif zip_file_path.endswith(('.tar', '.tar.gz', '.tgz')):
    with tarfile.open(zip_file_path, 'r') as tar_ref:
        tar_ref.extractall(destination_dir)
else:
    print(f"Unsupported file extension: {zip_file_path}")

In [8]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] > 0.95 and logs['val_accuracy'] > 0.7:
            print("\nReached desired accuracy, stopping training.")
            self.model.stop_training = True

In [9]:
data_directory = '/content/images/datasets_character_anime'

In [10]:
image_filenames = [
    os.path.join(data_directory, folder, filename)
    for folder in os.listdir(data_directory)
    if os.path.isdir(os.path.join(data_directory, folder))
    for filename in os.listdir(os.path.join(data_directory, folder))
    if filename.endswith(('.jpg', '.png', '.jpeg'))
]

In [11]:
for filename in image_filenames:
    img = Image.open(filename)
    img = img.resize((128, 128))
    img.save(filename)

batch_size = 32
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1
)

In [15]:
class_names = list(train_generator.class_indices.keys())
print("Class names:", class_names)

Class names: ['Anya_Forger', 'Aquamarine_Hoshino', 'Ayaka_Genshin_Impact', 'Boa_Hancock', 'Charlotte_Genshin_Impact', 'Damian_Desmond', 'Dazai_Osamu_BSD', 'Ganyu_genshin', 'Gojo_Satoru', 'Jett_Valorant', 'Kafka_Honkai_Star_Rail', 'Keqing_genshin', 'Killjoy_Valorant', 'Kobo_kanaeru', 'Kugisaki_Nobara', 'Loid_Forger', 'Luffy_D_Monkey', 'Midoriya_Izuku', 'Minato_Aqua', 'Misa_Amane', 'Mitsuri_Kanroji', 'Nico_Robin', 'Nier_Automata_9S', 'Nier_Automata_A2', 'Raiden_Shogun_Genshin_Impact', 'Ruby_Hoshino', 'Sakura_Haruno', 'Todoroki_Shoto', 'Tokisaki_Kurumi', 'Uraraka_Ochako', 'Wanderer', 'Yor_Forger', 'Yuri_Briar', 'Zerotwo', 'Zoro', 'amelia_watson', 'arima_kousei', 'gawr_gura', 'hoshino_ai', 'hutao_genshin', 'mikasa', 'tanjiro', 'violet_evergarden', 'zeta_hololive']


In [12]:
num_classes = 44

In [13]:
train_generator = datagen.flow_from_directory(
    data_directory,
    target_size=(128, 128),
    class_mode='categorical',
    subset='training',
    shuffle=True,
    batch_size=batch_size
)

Found 7302 images belonging to 44 classes.


In [14]:
validation_generator = datagen.flow_from_directory(
    data_directory,
    target_size=(128, 128),
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    batch_size=batch_size
)

Found 791 images belonging to 44 classes.


In [16]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=num_classes, activation='softmax'))

In [17]:
custom_optimizer = Adam(learning_rate=0.001)

In [18]:
model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
num_epochs = 100
custom_callback = CustomCallback()

model.fit(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=None,
    validation_steps=None,
    callbacks=[custom_callback],
    epochs=num_epochs
)

model.save('new_anime_classificationL.h5')

Epoch 1/100
 50/229 [=====>........................] - ETA: 1:00 - loss: 3.7641 - accuracy: 0.0254

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


229/229 [==============================] - 89s 382ms/step - loss: 3.6220 - accuracy: 0.0619 - val_loss: 3.3535 - val_accuracy: 0.1315
Epoch 2/100
229/229 [==============================] - 85s 373ms/step - loss: 3.2979 - accuracy: 0.1223 - val_loss: 2.9981 - val_accuracy: 0.1884
Epoch 3/100
229/229 [==============================] - 86s 374ms/step - loss: 3.0855 - accuracy: 0.1667 - val_loss: 2.8430 - val_accuracy: 0.2339
Epoch 4/100
229/229 [==============================] - 85s 373ms/step - loss: 2.9587 - accuracy: 0.1897 - val_loss: 2.7798 - val_accuracy: 0.2617
Epoch 5/100
229/229 [==============================] - 85s 368ms/step - loss: 2.8380 - accuracy: 0.2102 - val_loss: 2.6321 - val_accuracy: 0.2832
Epoch 6/100
229/229 [==============================] - 84s 365ms/step - loss: 2.7404 - accuracy: 0.2438 - val_loss: 2.4475 - val_accuracy: 0.2756
Epoch 7/100
229/229 [==============================] - 81s 353ms/step - loss: 2.6050 - accuracy: 0.2757 - val_loss: 2.2982 - val_accurac

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# Load the trained model
model_path = 'new_anime_classificationL'  # Replace with the actual path to your model
loaded_model = load_model(model_path)

# Function to preprocess the input image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array / 255.0

# Function to make predictions
def predict_image(model, img_path):
    processed_image = preprocess_image(img_path)
    prediction = model.predict(processed_image)
    return prediction

# Replace 'your_image.jpg' with the actual path to the image you want to test
image_path = 'misa4.jpg'
result = predict_image(loaded_model, image_path)
# Assuming you have 44 classes, replace this with your actual class names
class_names = ['Anya_Forger', 'Aquamarine_Hoshino', 'Ayaka_Genshin_Impact', 'Boa_Hancock', 'Charlotte_Genshin_Impact', 'Damian_Desmond', 'Dazai_Osamu_BSD', 'Ganyu_genshin', 'Gojo_Satoru', 'Jett_Valorant', 'Kafka_Honkai_Star_Rail', 'Keqing_genshin', 'Killjoy_Valorant', 'Kobo_kanaeru', 'Kugisaki_Nobara', 'Loid_Forger', 'Luffy_D_Monkey', 'Midoriya_Izuku', 'Minato_Aqua', 'Misa_Amane', 'Mitsuri_Kanroji', 'Nico_Robin', 'Nier_Automata_9S', 'Nier_Automata_A2', 'Raiden_Shogun_Genshin_Impact', 'Ruby_Hoshino', 'Sakura_Haruno', 'Todoroki_Shoto', 'Tokisaki_Kurumi', 'Uraraka_Ochako', 'Wanderer', 'Yor_Forger', 'Yuri_Briar', 'Zerotwo', 'Zoro', 'amelia_watson', 'arima_kousei', 'gawr_gura', 'hoshino_ai', 'hutao_genshin', 'mikasa', 'tanjiro', 'violet_evergarden', 'zeta_hololive']

# Get the predicted class index
predicted_class_index = np.argmax(result)
predicted_class = class_names[predicted_class_index]

print(f"The predicted class is: {predicted_class}")